In [1]:
print("Kernel is up")

Kernel is up


In [2]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
tf.enable_eager_execution()

import librosa
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import librosa
#import simpleaudio as sa
import math
from IPython import display
import sys

print("Finished imports")

Finished imports


In [3]:
# Converting into spectrograms
# Spectrograms are stored into spectrograms.tfrecord
nsynth = iter(tf.data.TFRecordDataset("gs://jz-datasets/nsynth/train.tfrecord"))

def acousticOnly():
    i = 0
    just_skipped = False
    while(True):
        try: 
            data = next(nsynth)
        except:
            break
        data = tf.parse_single_example(serialized=data, features=read_features)
        if(data['instrument_source'].numpy()==0):
            if(just_skipped):
                print("Skipped {}".format(i))
            just_skipped = False
            yield data
        else:
            if(not(just_skipped)):
                print("                                       ", end='\r')
            i += 1
            if(i % 500 == 0):
                print("Skipped {}".format(i), end="\r")
            just_skipped = True

record = acousticOnly()

read_features = {
    'note': tf.FixedLenFeature([], dtype=tf.int64),
    'note_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_str': tf.FixedLenFeature([], dtype=tf.string),
    'pitch': tf.FixedLenFeature([], dtype=tf.int64),
    'velocity': tf.FixedLenFeature([], dtype=tf.int64),
    'sample_rate': tf.FixedLenFeature([], dtype=tf.int64),
    'audio': tf.VarLenFeature(dtype=float),
    'instrument_family': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_family_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument_source': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_source_str': tf.FixedLenFeature([], dtype=tf.string)
}

# 2919 per epoch
start_on_file = 20
for i in range(2919*start_on_file):
    if(i % 200 == 0):
        print("Skipped {} out of {}".format(i, 2919*start_on_file), end='\r')
    next(record)
if(start_on_file != 0):
    print("Skipped {} out of {}".format(2919*start_on_file, 2919*start_on_file))
    
for i in range(35):
    if(i < start_on_file):
        continue
    print("Starting file {}".format(i))
    with tf.python_io.TFRecordWriter('gs://jz-datasets/spec-files/spectrograms{}.tfrecord'.format(i)) as writer:
        total = 0
        for _ in range(2919):
            total += 1
            data = next(record)
            audio = data['audio'].values.numpy()*15000
            audio = np.array(audio)
            spec = librosa.stft(audio, 254, 127)
            magnitudes = np.log(np.abs(spec) + 1.2664166e-14).reshape(1, 128, 504)
            phases = np.angle(spec).reshape(1, 128, 504)
            magnitudes = 2*(magnitudes+32)/(16+32)-1
            spec = np.concatenate([magnitudes, tf.cos(phases), tf.sin(phases)], 0).flatten().tolist()
            example = tf.train.Example(features=tf.train.Features(feature={
                'note': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['note'].numpy()])),
                'note_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['note_str'].numpy()])),
                'instrument': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument'].numpy()])),
                'instrument_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_str'].numpy()])),
                'pitch': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['pitch'].numpy()])),
                'velocity': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['velocity'].numpy()])),
                'sample_rate': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['sample_rate'].numpy()])),
                'spectrogram': tf.train.Feature(float_list=tf.train.FloatList(value=spec)),
                'instrument_family': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_family'].numpy()])),
                'instrument_family_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_family_str'].numpy()])),
                'instrument_source': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_source'].numpy()])),
                'instrument_source_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_source_str'].numpy()]))
            }))

            writer.write(example.SerializeToString())
            if(total % 20 == 0):
                print("Finished {} out of {}".format(total, 2919), end='\r')
        print()
            
print("\nDone")

Skipped 57517                          
Skipped 68476                          
Skipped 71111                          
Skipped 72871                          
Skipped 73311                          
Skipped 76391                          
Skipped 76831                          
Skipped 77711                          
Skipped 78151                          
Skipped 78591                          
Skipped 79471                          
Skipped 79911                          
Skipped 80351                          
Skipped 81231                          
Skipped 81884                          
Skipped 82412                          
Skipped 82852                          
Skipped 86593                          
Skipped 89884                          
Skipped 105724                         
Skipped 124016                         
Skipped 130264                         
Skipped 58380 out of 58380
Starting file 20
Finished 2900 out of 2919
Starting file 21
Finished 2900 out of 2919
Startin

In [16]:
!rm .lock

rm: cannot remove '.lock': No such file or directory
